In [1]:
import numpy as np 
import pandas as pd
import math

data=pd.read_csv('seeds_dataset.csv')
data.head()

,A,B,C,D,E,F,G,T
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [2]:
def normalise_scale(y,a,b):
    l=[]
    for i in range(len(y)):
        l.append(((y[i]-min(y))/(max(y)-min(y)))*(a-b)+b)
        
    return l  

target=np.asarray(data['T'])
data=np.asarray(data.drop('T',axis=1))
for i in range(len(data[0])):
    data[:,i]=normalise_scale(data[:,i],0.999,.001)

print(data)

[[0.44110009 0.50206198 0.57063884 ... 0.48612901 0.18992304 0.34545987]
 [0.40528895 0.44638843 0.66210708 ... 0.501067   0.03381725 0.21573461]
 [0.34968744 0.34741322 0.87855172 ... 0.50391233 0.25195011 0.15136337]
 ...
 [0.24696601 0.25874793 0.72731216 ... 0.42922238 0.98070331 0.26487297]
 [0.11879981 0.16595868 0.3994755  ... 0.14753457 0.36860772 0.25897637]
 [0.16215014 0.19276446 0.54709256 ... 0.2456985  0.633196   0.26831265]]


In [3]:
def cost_func(ypred,y):
    return -np.sum(np.asarray([math.log(ypred[x],(2)) if y[x]==1 else 0 for x in range(len(y)) ]))

def softmax(z):
    return np.exp(z)/np.sum(np.exp(z))

def sigmoid(z):
    sig=1/(1+np.exp(-z))
    sig=np.minimum(sig, 0.9999)
    sig=np.maximum(sig, 0.0001)
    return sig

def initialize_weights(inp_neurons,out_neurons):
    l=[]
    for i in range(out_neurons):
        W=np.random.randn(inp_neurons)
        l.append(W)
    b=np.zeros(out_neurons)
    return l,b

def normalise_scale(y,a,b):
    l=[]
    for i in range(len(y)):
        l.append(((y[i]-min(y))/(max(y)-min(y)))*(a-b)+b)
        
    return l  
    
def forwardpropogate(totalW,totalB,data,n):
    ffout=[]
    ffout.append(data)
    for i in range(0,n-1):
        params=[]
        for j in range(len(totalW[i])):
            
            ###############  W(transpose).X + B  ###############
            
            params.append(np.dot(np.asarray(totalW[i][j]).reshape(1,len(totalW[i][j])),np.asarray(data).reshape(len(data),1)).reshape(1)[0]+totalB[i][j])
        
        data=params
        if(i!=n-2):
            ##################  activated  ###############
    
            data=sigmoid(np.asarray(data))
            ffout.append(data)
        
    ##################  final_layer-activated  ###############
    
    ffout.append(softmax(np.asarray(data)))
    return ffout

def backwardpropogate(totalW,ffout,y,n):
    totalgradW=[]
    totalgradB=[]
    
    ##################  final_layer-gradient  ###############
    
    grad_wrt_out=-(np.asarray(y)-np.asarray(ffout[len(ffout)-1]))
    grad_inter=grad_wrt_out
    
    for i in range(n-2,-1,-1):
        
        ##################  gradW = (gradient_at_layer) . (activated_output_previous_layer)  ###############
        
        totalgradW.insert(0,np.dot(np.asarray(grad_inter).reshape(len(grad_inter),1),np.asarray(ffout[i]).reshape(1,len(ffout[i]))).reshape(len(grad_inter),len(ffout[i])))
        ##################  gradB = (gradient_at_layer)  ###############
        
        totalgradB.insert(0,grad_inter)
        grad_pre=[]
        
        ##################  grad_at_previous_layer = (gradient_at_layer) . (weight_matrix)  ###############
        
        for j in range(len(totalW[i][0])):
            grad_pre.append(np.dot(np.asarray(grad_inter).reshape(1,len(grad_inter)),np.asarray(np.matrix(totalW[i])[:,j])).reshape(1)[0])
        
        ####  grad_at_previous_layer = (grad_at_previous_layer) . (hadamard) . (derivative_of_activation)  ####
        
        grad_inter=np.asarray(grad_pre)*(np.asarray(ffout[i]))*(1-np.asarray(ffout[i]))
    
    return totalgradW,totalgradB
                   
    
totalW=[]
totalB=[]
max_iterations=50
w,b=initialize_weights(len(data[0]),5)
totalW.append(w)
totalB.append(b)
w,b=initialize_weights(5,3)
totalW.append(w)
totalB.append(b)
#w,b=initialize_weights(3,3)
#totalW.append(w)
#totalB.append(b)
no_layers=3

cnt=0
while(cnt<max_iterations):
    cnt+=1
    for i in range(len(data)):
        y_one_hot=[1 if x==target[i] else 0 for x in set(target)]
        print(y_one_hot)
     
        ffout=forwardpropogate(totalW,totalB,data[i],no_layers)
        C=cost_func(ffout[len(ffout)-1],y_one_hot)
        print('loss=======',C)
        print('value obtained :',ffout[len(ffout)-1])
        
        gradW,gradB=backwardpropogate(totalW,ffout,y_one_hot,no_layers)
        
        if(i==0):
            totalgradW1=gradW
            totalgradB1=gradB
        else:
            for j in range(len(gradW)):
                totalgradW1[j]=np.asarray(totalgradW1[j])+np.asarray(gradW[j])
                totalgradB1[j]=np.asarray(totalgradB1[j])+np.asarray(gradB[j])
        if((i+1) % 32 == 0):
            for i in range(len(totalW)):
                totalW[i]=np.asarray(totalW[i])-(0.05*np.asarray(totalgradW1[i]))
                totalB[i]-=0.05*totalgradB1[i]
            
            for j in range(len(gradW)):
                totalgradW1[j]=0
                totalgradB1[j]=0
                
            print()

    print()
    

ss======= 0.06975247232205228
value obtained : [0.95280146 0.01172253 0.03547601]
[1, 0, 0]
loss======= 0.08719569289302298
value obtained : [0.94135077 0.01304292 0.04560631]
[1, 0, 0]
loss======= 0.07804669201350697
value obtained : [0.94733941 0.01063486 0.04202573]
[1, 0, 0]
loss======= 0.05786252235644171
value obtained : [0.96068641 0.01732201 0.02199159]
[1, 0, 0]
loss======= 0.0649667567278525
value obtained : [0.95596735 0.01426583 0.02976683]
[1, 0, 0]
loss======= 0.053205242517142565
value obtained : [0.96379269 0.02889364 0.00731367]
[1, 0, 0]
loss======= 0.011755063015239035
value obtained : [0.99188512 0.00371954 0.00439535]
[1, 0, 0]
loss======= 1.6521477137771694
value obtained : [0.31816616 0.00326203 0.67857181]
[1, 0, 0]
loss======= 0.29163299101897705
value obtained : [0.8169768 0.0555166 0.1275066]
[1, 0, 0]
loss======= 0.15801000565949924
value obtained : [0.89626048 0.04246686 0.06127265]
[1, 0, 0]
loss======= 0.060236039316358014
value obtained : [0.95910719 0.0

In [5]:
ffout=forwardpropogate(totalW,totalB,data[200],no_layers)
y_one_hot=[1 if x==target[200] else 0 for x in set(target)]
print('Expected Output :',y_one_hot)
print('Predicted Output :',ffout[len(ffout)-1])

Expected Output : [0, 0, 1]
Predicted Output : [1.83075924e-02 2.60963429e-04 9.81431444e-01]
